In [ ]:
pip install pandas
pip install nltk
pip install bert-score

## Prepration


### Importing libraries and data

In [1]:
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
import nltk
from bert_score import score

In [28]:

data = pd.read_csv("fusion_rag_model_final_eval.csv")
data = pd.read_csv("naive_rag_model_final_eval.csv")
# data = pd.read_csv("baseline_model_eval_final.csv")
# data = pd.read_csv("vinallama.csv")
# data = pd.read_csv("phogpt_answers.csv")
#data = pd.read_csv("vistral-legal-chat.csv")

In [35]:
nltk.download('punkt')  # Ensure that the tokenizer models are downloaded

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
data.tail()

,Question,Real answer,Bot Answer (naive RAG),precision,recall,f1
94,Những ai không được quyền hưởng di sản theo đi...,Người bị kết án về hành vi cố ý xâm phạm tính ...,Những người bị kết án về hành vi cố ý xâm phạm...,0.784399,0.891905,0.834705
95,Trường hợp không có người thừa kế theo di chúc...,Tài sản còn lại sau khi đã thực hiện nghĩa vụ ...,Theo quy định tại Điều 622 Bộ luật Dân sự năm ...,NaN,NaN,NaN
96,Có qui định gì tuổi tác của người lập di chúc?,Người từ đủ mười lăm tuổi đến chưa đủ mười tám...,- Người thành niên có đủ điều kiện theo quy đị...,0.591299,0.704200,0.697690
97,"Trong thời hạn bảo hành, nếu bên mua phát hiện...","Có. Trong thời hạn bảo hành, nếu bên mua phát ...",Dựa vào Điều 447 Bộ luật Dân sự 2015 quy định ...,0.615675,0.809503,0.712577
98,Đối với hợp đồng vay không kỳ hạn và không có ...,Đối với hợp đồng vay không kỳ hạn và không có ...,"Theo Điều 469 Bộ luật dân sự năm 2015, đối với...",0.659492,0.881740,0.770474


### Evaluation using BertScore

Extracting 'Bot Answer' and 'Real answer' columns from the dataset.

'candidates' will hold all the answers provided by the bot, and 
'references' will hold the corresponding real or ground truth answers.

In [ ]:
candidates = [x for x in data['Bot Answer']]
references = [x for x in data['Real answer']]

Check of empty or nan data row

In [23]:
# Iterate through the candidates and references to check for float type or emptiness
for i, (candidate, reference) in enumerate(zip(candidates, references)):
    if isinstance(candidate, float) or candidate == "":
        print(candidate)
        print(f"Row {i}: Candidate is float or empty: {candidate}")
    if isinstance(reference, float) or reference == "":
        print(f"Row {i}: Reference is float or empty: {reference}")


Calculating the precision, recall, and F1 score for the bot's answers.

The 'score' function compares 'candidates' (bot's answers) with 'references' (real answers).

The 'lang' parameter is set to 'vi' for Vietnamese language processing.

'verbose=True' enables detailed output during the scoring process.

precision, recall, f1 = score(candidates, references, lang='vi', verbose=True)

In [ ]:

precision, recall, f1 = score(candidates, references, lang='vi', verbose=True)

This script processes and evaluates the performance of a model's answers.

It starts by extracting the 'Bot Answer' and 'Real answer' columns from a dataset into lists.

These lists are then passed to a scoring function that calculates precision, recall, and F1 score.

The resulting tensor scores are converted to lists and organized into a pandas DataFrame.

Finally, the DataFrame is saved as a CSV output file containing the results with the data separated by semicolons and without including the index.


In [17]:
precision_list = precision.tolist()
recall_list = recall.tolist()
f1_list = f1.tolist()

df = pd.DataFrame({
    'precision': precision_list,
    'recall': recall_list,
    'f1': f1_list
})

df.to_csv('output .csv', index=False, sep=';')


### Evaluation using BLEU score

This script calculates the BLEU score for each candidate (bot's answer) compared to the corresponding reference (real answer).

The BLEU score is calculated using a specified weight configuration that focuses on the first two n-grams (unigram and bigram)
.
For each pair of candidate and reference, the text is tokenized into words.

The sentence-level BLEU score is then computed and appended to the 'bleu_scores' list.


In [ ]:
bleu_scores = []
weights = (0.25, 0.25, 0, 0)  
for i in range(len(candidates)):
    tokenized_candidate = nltk.word_tokenize(candidates[i])
    tokenized_reference = nltk.word_tokenize(references[i])
    
    bleu_score = sentence_bleu([tokenized_reference], tokenized_candidate, weights=weights)
    
   
    bleu_scores.append(bleu_score)
    

Create a DataFrame containing the calculated BLEU scores.

The DataFrame is then saved to an output CSV file.

The data is saved without the index and separated by a semicolon (';').

In [ ]:
df = pd.DataFrame({
    'bleu_scores': bleu_scores,
    
})

df.to_csv('output.csv', index=False, sep=';')
